# Exploratory data analysis

Initial review of `external` and `raw` data sources, data cleansing/scrubbing, and save to `interim` for use in model.

FIPS: Federal Information Processing Standards are numeric codes assigned by the National Institute of Standards and Technology (NIST). Typically, FIPS codes deal with US [states](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) and [counties](https://en.wikipedia.org/wiki/FIPS_county_code). US states are identified by a 2-digit number, while US counties are identified by a 3-digit number. Although this standard has been _withdrawn_, it is **still widely used in geospatial datasets**.

## Import relevant libraries

In [210]:
import json
import t4
import pandas as pd
import numpy as np
import datetime as dt

## Import & review topology data

In [211]:
topo_data = json.load(open("../data/external/us-10m.json"))

In [212]:
topo_df = pd.DataFrame(topo_data["objects"]["counties"])

In [213]:
topo_df.head()

,type,geometries
0,GeometryCollection,"{'type': 'MultiPolygon', 'arcs': [], 'id': 22051}"
1,GeometryCollection,"{'type': None, 'id': 23023}"
2,GeometryCollection,"{'type': None, 'id': 37031}"
3,GeometryCollection,"{'type': None, 'id': 42045}"
4,GeometryCollection,"{'type': 'Polygon', 'arcs': [[0, 1, 2, 3, 4, 5..."


### Drill down into geometry data to review FIPS format

In [214]:
geometries = topo_df['geometries'].apply(json.dumps)

In [215]:
geometries.head(10)

0    {"type": "MultiPolygon", "arcs": [], "id": 22051}
1                          {"type": null, "id": 23023}
2                          {"type": null, "id": 37031}
3                          {"type": null, "id": 42045}
4    {"type": "Polygon", "arcs": [[0, 1, 2, 3, 4, 5...
5    {"type": "MultiPolygon", "arcs": [[[95, 96, -2...
6    {"type": "Polygon", "arcs": [[98, 99, 100, 101...
7    {"type": "Polygon", "arcs": [[104, 105, 106, 1...
8    {"type": "Polygon", "arcs": [[114, 115, 116, 1...
9    {"type": "Polygon", "arcs": [[-103, 118, 119, ...
Name: geometries, dtype: object

## Import and review raw poverty, smoking & opioid data

In [238]:
# Clean up
df = None
states_df = None
counties_df = None

In [239]:
df = pd.read_csv("../data/raw/poverty_smoking_opioid.csv")

In [240]:
df.head()

,geo_name,geo_sumlevel,geo_id,adult_smoking_2015,adult_smoking_2016,adult_smoking_2017,income_below_poverty_2013,income_below_poverty_2014,income_below_poverty_2015,income_below_poverty_2016,...,opioid_overdose_deathrate_ageadjusted_2011,opioid_overdose_deathrate_ageadjusted_2012,opioid_overdose_deathrate_ageadjusted_2013,opioid_overdose_deathrate_ageadjusted_2014,opioid_overdose_deathrate_ageadjusted_2015,opioid_overdose_deathrate_ageadjusted_2016,pop_poverty_status_2013,pop_poverty_status_2014,pop_poverty_status_2015,pop_poverty_status_2016
0,Alabama,state,04000US01,0.219,0.211,0.214,870631.0,889710.0,887260.0,868666.0,...,3.8,3.6,3.5,5.6,6.1,7.5,4682980.0,4699510.0,4711440.0,4720590.0
1,Alaska,state,04000US02,0.208,0.199,0.191,69514.0,71866.0,72957.0,72826.0,...,8.7,10.8,9.2,10.6,11.0,12.5,704041.0,711235.0,716218.0,719064.0
2,Arizona,state,04000US04,0.166,0.165,0.140,1131900.0,1169310.0,1180690.0,1165640.0,...,8.8,8.2,8.2,8.8,10.2,11.4,6333790.0,6411350.0,6488920.0,6573520.0
3,Arkansas,state,04000US05,0.224,0.247,0.249,547328.0,549303.0,553644.0,542431.0,...,6.2,6.0,5.6,6.3,7.2,5.9,2849640.0,2862660.0,2872500.0,2881400.0
4,California,state,04000US06,0.129,0.128,0.117,5885420.0,6115240.0,6135140.0,6004260.0,...,5.0,4.4,4.9,5.0,4.9,4.9,36913400.0,37323100.0,37678700.0,37913100.0


In [241]:
df.geo_sumlevel.unique()

array(['state', 'county', nan], dtype=object)

## Data cleansing steps

### 1. Generate FIPS `id` code from `geo_id` field

In [242]:
# Generate FIPS code
f = lambda x: x["geo_id"].split("US")[1]
df.loc[:, 'id'] = df.apply(f, axis=1)

### 2. Save state-grain data to JSON file

In [243]:
states_df = df.loc[df["geo_sumlevel"] == "state", :].copy()

In [244]:
states_df.to_json(
    "../data/interim/states_opioid.json",
    orient='records'
)

### 3. Subset dataset to county-grain

In [245]:
counties_df = df.loc[df.geo_sumlevel == "county", :].copy()

### 4. Ensure county FIPS `id` code is an integer and not a float

In [246]:
f_int = lambda x: int(x["id"])
counties_df.loc[:, 'id'] = counties_df.apply(f_int, axis=1)

### 5. Generate state value from `geo_name` field

In [247]:
# Generate state code
f_state = lambda x: x["geo_name"].split(", ")[1]
counties_df.loc[:, 'state'] = counties_df.apply(f_state, axis=1)

### 6. Drop columns with NaNs

In [248]:
counties_df.dropna(
    axis='columns',
    how='all',
    inplace=True
)

### 7. Generate percent totals for 2015 & 2016

In [249]:
f_percent_2015 = lambda x: x["income_below_poverty_2015"] / x["pop_poverty_status_2015"]
counties_df["population_in_poverty_percent_2015"] = counties_df.apply(f_percent_2015, axis=1)

In [250]:
f_percent_2016 = lambda x: x["income_below_poverty_2016"] / x["pop_poverty_status_2016"]
counties_df["population_in_poverty_percent_2016"] = counties_df.apply(f_percent_2016, axis=1)

### 8. Drop unused columns

In [251]:
counties_df.drop(
    [
        'geo_sumlevel',
        'geo_id',
        'adult_smoking_2017',
        'income_below_poverty_2013',
        'income_below_poverty_2014',
        "income_below_poverty_2015",
        "income_below_poverty_2016",
        'pop_poverty_status_2013',
        'pop_poverty_status_2014',
        "pop_poverty_status_2015",
        "pop_poverty_status_2016"
    ],
    axis=1,
    inplace=True
)

### 9. Rename columns for clarity 

In [252]:
counties_df.rename(
    index=str,
    columns={
        "adult_smoking_2015": "smoking_2015",
        "adult_smoking_2016": "smoking_2016",
        "population_in_poverty_percent_2015": "poverty_2015",
        "population_in_poverty_percent_2016": "poverty_2016"
    },
    inplace=True
)

## Review data

Useful for setting scales in Vega specifications.

In [253]:
counties_df.head()

,geo_name,smoking_2015,smoking_2016,id,state,poverty_2015,poverty_2016
52,"Autauga County, AL",0.217,0.187,1001,AL,0.128794,0.122660
53,"Baldwin County, AL",0.206,0.186,1003,AL,0.134117,0.129938
54,"Barbour County, AL",0.251,0.214,1005,AL,0.267274,0.263737
55,"Bibb County, AL",0.259,0.210,1007,AL,0.167959,0.164539
56,"Blount County, AL",0.213,0.194,1009,AL,0.167209,0.165344


In [254]:
counties_df.describe()

,smoking_2015,smoking_2016,id,poverty_2015,poverty_2016
count,2708.000000,3137.000000,3223.000000,3220.000000,3220.000000
mean,0.212777,0.183957,31395.379150,0.174926,0.171960
std,0.063144,0.037886,16298.491965,0.083184,0.082883
min,0.031000,0.069000,1001.000000,0.014245,0.018104
25%,0.170000,0.157000,19032.000000,0.121000,0.118344
50%,0.208000,0.178000,30025.000000,0.161547,0.158750
75%,0.249000,0.207000,46108.000000,0.206637,0.203294
max,0.511000,0.412000,72153.000000,0.641964,0.637711


## Save scrubbed data to CSV file

In [255]:
counties_df.to_csv(
    "../data/interim/counties_smoking_poverty_2015-2016.csv",
    index=False
)

## Instantiate local T4 package, add data, & build

In [51]:
p = t4.Package()

In [256]:
# Add generated cleansed/scrubbed dataset
p = p.set(
    "data/interim/counties_smoking_poverty_2015-2016.csv",
    "../data/interim/counties_smoking_poverty_2015-2016.csv",
    meta=dict(
        data_source="DataUSA",
        data_source_url="https://datausa.io",
        data_source_description="County-level smoking and \
        poverty data from 2015 and 2016",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="csv"
    )
)

In [182]:
# Add US topology dataset for Vega specification
p = p.set(
    "data/external/us-10m.json",
    "../data/external/us-10m.json",
    meta=dict(
        data_source="TopoJSON",
        data_source_url="https://github.com/topojson/us-atlas",
        data_source_description="Pre-built TopoJSON from the U.S. Census Bureau",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="json"
    )
)

In [314]:
# Remove unused Vega specification file
p = p.delete("src/visualization/vega_specs/states.json")

In [201]:
# Vega specification file for poverty 2015
p = p.set(
    "src/visualization/vega_specs/us-county-poverty-2015.json",
    "../src/visualization/vega_specs/us-county-poverty-2015.json",
    meta=dict(
        description="Vega specification of US county percent population in poverty in 2015",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="json"
    )
)

In [196]:
# Vega specification file for smoking 2015
p = p.set(
    "src/visualization/vega_specs/us-county-smoking-2015.json",
    "../src/visualization/vega_specs/us-county-smoking-2015.json",
    meta=dict(
        description="Vega specification of US county percent population smoking in 2015",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="json"
    )
)

In [346]:
# Vega specification for interactive scatter plot of 2015 smoking vs. population
p = p.set(
    "src/visualization/vega_specs/us-county-poverty-vs-smoking-2015.json",
    "../src/visualization/vega_specs/us-county-poverty-vs-smoking-2015.json",
    meta=dict(
        description="Vega interactive scatterplot specification \
        of US county percent population smoking vs poverty in 2015",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="json"
    )
)

In [325]:
# Add our project description markdown file
p = p.set(
    "description.md",
    "../description.md",
    meta=dict(
        description="Markdown (.md) file \
        to describe project",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="md"
    )
)

In [316]:
# Add EDA notebook
p = p.set(
    "notebooks/summarize_exploratory_data_analysis.ipynb",
    "summarize_exploratory_data_analysis.ipynb",
    meta=dict(
        description="Exploratory data analysis notebook",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="ipynb"
    )

)

In [317]:
# Add Quilt summarize file
p = p.set(
    "quilt_summarize.json",
    "../quilt_summarize.json",
    meta=dict(
        description="Quilt summarize file \
        for auto-visualization of datasets",
        last_updated=str(dt.datetime.now()),
        uploaded_by="robnewman",
        project="us-county-smoking-poverty-2015-2016",
        filetype="json"
    )

)

In [347]:
tophash = p.build("robnewman/us-county-smoking-vs-poverty")

In [348]:
# Successfully built package
tophash

'1f444fefa7dc7fcf8148059fb530b4c1ba85b71368b3bd50b968e97a97ce3f33'

In [349]:
p.push(
    "robnewman/us-county-smoking-vs-poverty",
    "s3://alpha-quilt-storage/us-county-smoking-vs-poverty"
)

data/
  external/
    us-10m.json
  interim/
    counties_smoking_poverty_2015-2016.csv
description.md
notebooks/
  summarize_exploratory_data_analysis.ipynb
quilt_summarize.json
src/
  visualization/
    vega_specs/
      us-county-poverty-2015.json
      us-county-poverty-vs-smoking-2015.json
      us-county-smoking-2015.json